In [1]:
from os import listdir
import torch
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
from PIL import Image, ImageFile
import random
import pickle
from torchvision import transforms
import re


def transform_image(dir_path, name):

    img_name = dir_path + '/' + name + '.npy'

    images0 = torch.from_numpy(np.load(img_name)[:, 0, ...])

    images1 = torch.from_numpy(np.load(img_name)[:, 1, ...])
    
    images = images0 + images1

    trans = transforms.Compose([
                      
                      transforms.Lambda(lambda x: x.repeat(3, 1, 1) if x.size(0)==1 else x)])
    #transforms.Resize(256),

    images_1 = [trans(images[i].unsqueeze(0)/1.) for i in range(images.shape[0])]

    images2 = np.stack(images_1)

    images3 = torch.Tensor(images2.transpose((1, 2, 3, 0)))

    return images3


In [9]:
from sklearn.metrics import pairwise_distances
import numpy as np
import torchvision.models as models
from torch.nn import functional as f

model_ft = models.resnet50(pretrained=True)
feature_extractor = torch.nn.Sequential(*list(model_ft.children())[:-1])

def get_cosine_distance(im_url1, im_url2):
    image1 = feature_extractor(im_url1.unsqueeze(0))
    image2 = feature_extractor(im_url2.unsqueeze(0))
    #.reshape(1, -1)
    return f.cosine_similarity(image1, image2)

In [3]:
#get names

from os import listdir
dir_path = '/data/burenko/datasets/pet-ct'
names = []
for dir_content in listdir(dir_path):
    if dir_content.split('.')[-1] == 'npy':
        names.append(dir_content.split('.')[0])

In [10]:
#
from tqdm import tqdm

vec_sims = []
for name in tqdm(names):
    images = transform_image(dir_path, name)
    similarities = [0.]
    for i in range(images.shape[3] - 1):
        vec_i = images[..., i]
        vec_j = images[..., i + 1]
        similarities.append(get_cosine_distance(vec_i, vec_j))
    similarities.append(0.)
    vec_sims.append(similarities)

  0%|          | 0/183 [03:39<?, ?it/s]


KeyboardInterrupt: 

In [11]:
similarities

[0.0,
 tensor([[[0.9995]]], grad_fn=<DivBackward0>),
 tensor([[[0.9996]]], grad_fn=<DivBackward0>),
 tensor([[[0.9995]]], grad_fn=<DivBackward0>),
 tensor([[[0.9996]]], grad_fn=<DivBackward0>),
 tensor([[[0.9996]]], grad_fn=<DivBackward0>),
 tensor([[[0.9996]]], grad_fn=<DivBackward0>),
 tensor([[[0.9996]]], grad_fn=<DivBackward0>),
 tensor([[[0.9997]]], grad_fn=<DivBackward0>),
 tensor([[[0.9996]]], grad_fn=<DivBackward0>),
 tensor([[[0.9997]]], grad_fn=<DivBackward0>),
 tensor([[[0.9997]]], grad_fn=<DivBackward0>),
 tensor([[[0.9996]]], grad_fn=<DivBackward0>),
 tensor([[[0.9997]]], grad_fn=<DivBackward0>),
 tensor([[[0.9997]]], grad_fn=<DivBackward0>),
 tensor([[[0.9996]]], grad_fn=<DivBackward0>),
 tensor([[[0.9997]]], grad_fn=<DivBackward0>),
 tensor([[[0.9996]]], grad_fn=<DivBackward0>),
 tensor([[[0.9996]]], grad_fn=<DivBackward0>),
 tensor([[[0.9996]]], grad_fn=<DivBackward0>),
 tensor([[[0.9996]]], grad_fn=<DivBackward0>),
 tensor([[[0.9995]]], grad_fn=<DivBackward0>),
 tensor

In [22]:
vec_sims[0]

IndexError: list index out of range

In [16]:
[np.mean(vec_sims[i]) for i in range(len(vec_sims))]

/data/proskuryakova/.local/lib/python3.6/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


[array([[0.99104285]], dtype=float32),
 array([[0.99203336]], dtype=float32),
 array([[0.9920585]], dtype=float32),
 array([[0.9920886]], dtype=float32),
 array([[0.99213177]], dtype=float32)]

In [12]:
vec_sims[-1]

[0.0,
 array([[0.9977756]], dtype=float32),
 array([[0.9978779]], dtype=float32),
 array([[0.99790794]], dtype=float32),
 array([[0.9979167]], dtype=float32),
 array([[0.9980987]], dtype=float32),
 array([[0.99808085]], dtype=float32),
 array([[0.99813205]], dtype=float32),
 array([[0.998134]], dtype=float32),
 array([[0.9981159]], dtype=float32),
 array([[0.9981288]], dtype=float32),
 array([[0.99799365]], dtype=float32),
 array([[0.9980465]], dtype=float32),
 array([[0.99811137]], dtype=float32),
 array([[0.9980302]], dtype=float32),
 array([[0.99805677]], dtype=float32),
 array([[0.9981699]], dtype=float32),
 array([[0.9980999]], dtype=float32),
 array([[0.9981171]], dtype=float32),
 array([[0.99814224]], dtype=float32),
 array([[0.9982377]], dtype=float32),
 array([[0.9982022]], dtype=float32),
 array([[0.9980942]], dtype=float32),
 array([[0.99797624]], dtype=float32),
 array([[0.99802274]], dtype=float32),
 array([[0.99814427]], dtype=float32),
 array([[0.9980371]], dtype=float32